# Description:
        Implement sparsity filter
        Prune non-samples and 2 ECHCG samples that were not samples
        Final threshold: physeq.thresh = filter_taxa(physeq, 
                 function(x) sum(x > 3) > 3, TRUE)
        Join plant growth and enzyme metadata
        Save output for consistent use in other notebooks

# Setting variables

In [25]:
workDir = '/home/bryan/RhizCG/data/MiSeq_amplicon/MergedRuns/physeq/'
biomFileDir = '/home/bryan/RhizCG/data/MiSeq_amplicon/MergedRuns/OTU_binning/'

biomFile = '/home/bryan/RhizCG/data/MiSeq_amplicon/MergedRuns/OTU_binning/otu_table_wtax.biom'
metadataFile = '/home/bryan/RhizCG/data/MiSeq_amplicon/metadata_RhizCG_merged.txt'
treeFile = '/home/bryan/RhizCG/data/MiSeq_amplicon/MergedRuns/fasttree/otusn.tree'

# Init

In [26]:
import os
from pandas import DataFrame
from IPython.display import FileLink

In [27]:

%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [28]:
%%R
library(vegan)
library(phyloseq)
library(ggplot2)
library(foreach)
library(doParallel)
library(gridExtra)
library(dplyr)
library(tidyr)

# Loading/editting OTU table

In [29]:
%%R -i biomFile -i metadataFile -i workDir -i treeFile
# loading
setwd(workDir)

## biom file
physeq = import_biom(biomFile, treeFile)

## loading sample metadata
sample.data = import_qiime_sample_data(metadataFile)
physeq = merge_phyloseq(physeq,sample.data)



In [30]:
%%R
#remove controls
physeq = subset_samples(physeq, !is.na(Plant))

#remove two ECHCG plots where only soil and no roots were collected and sequenced
physeq = subset_samples(physeq, !(PlotID %in% c("4.16_Nneg", "4.16_Npos"))) 
print(physeq)


phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 11246 taxa and 238 samples ]
sample_data() Sample Data:       [ 238 samples by 21 sample variables ]
tax_table()   Taxonomy Table:    [ 11246 taxa by 8 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 11246 tips and 11245 internal nodes ]


In [31]:
%%R

sample_data(physeq)$Planted = ifelse(sample_data(physeq)$Plant == "BARE", "BARE", "Rhizosphere")
sample_data(physeq)$Planted

  [1] "BARE"        "BARE"        "BARE"        "BARE"        "BARE"       
  [6] "BARE"        "BARE"        "BARE"        "BARE"        "Rhizosphere"
 [11] "Rhizosphere" "BARE"        "Rhizosphere" "Rhizosphere" "Rhizosphere"
 [16] "BARE"        "Rhizosphere" "BARE"        "BARE"        "BARE"       
 [21] "BARE"        "Rhizosphere" "BARE"        "BARE"        "BARE"       
 [26] "Rhizosphere" "Rhizosphere" "BARE"        "Rhizosphere" "Rhizosphere"
 [31] "Rhizosphere" "Rhizosphere" "Rhizosphere" "Rhizosphere" "Rhizosphere"
 [36] "Rhizosphere" "BARE"        "BARE"        "Rhizosphere" "Rhizosphere"
 [41] "Rhizosphere" "Rhizosphere" "Rhizosphere" "BARE"        "BARE"       
 [46] "Rhizosphere" "Rhizosphere" "Rhizosphere" "Rhizosphere" "Rhizosphere"
 [51] "Rhizosphere" "BARE"        "Rhizosphere" "Rhizosphere" "Rhizosphere"
 [56] "BARE"        "BARE"        "Rhizosphere" "Rhizosphere" "Rhizosphere"
 [61] "Rhizosphere" "Rhizosphere" "BARE"        "Rhizosphere" "Rhizosphere"
 [66] "Rhizo

### apply gsub across columns replacing __ w/ "" reload tax_table to phyloseq object

In [32]:
%%R
t = tax_table(physeq)
t = apply(t, 2, function(y) gsub("__", "", y))
tax_table(physeq) = t

# Explore Filtering Functions

In [33]:
%%R 
physeq.thresh = filter_taxa(physeq, 
                 function(x) sum(x > 3) > 3, TRUE)
physeq.thresh

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 4982 taxa and 238 samples ]
sample_data() Sample Data:       [ 238 samples by 22 sample variables ]
tax_table()   Taxonomy Table:    [ 4982 taxa by 8 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 4982 tips and 4981 internal nodes ]


Filtering function
function(x): sum(x > 3) > 3, TRUE): 4982 taxa 


In [34]:
%%R
head(sample_data(physeq.thresh))

Sample Data:        [6 samples by 22 sample variables]:
                           X.sample Library Primer_Plate Primer_Well_ID
sample2-1.06_Npos sample2-1.06_Npos       1            1             H9
sample6-4.27_Npos sample6-4.27_Npos       1            2            F11
sample4-4.27_Nneg sample4-4.27_Nneg       1            1            B12
sample4-1.06_Nneg sample4-1.06_Nneg       1            1            E11
sample2-1.06_Nneg sample2-1.06_Nneg       1            1             G9
sample6-3.07_Nneg sample6-3.07_Nneg       1            2            C11
                  primerFR_ID_byPlate primerFR_ID_total barcode_ID    PlotID
sample2-1.06_Npos                  72                72         72 1.06_Npos
sample6-4.27_Npos                  86               182        182 4.27_Npos
sample4-4.27_Nneg                  90                90         90 4.27_Nneg
sample4-1.06_Nneg                  85                85         85 1.06_Nneg
sample2-1.06_Nneg                  71                71

## Join plant data for consistent meta-data set across notebooks

In [35]:
%%R
plantData = read.table('/home/bryan/RhizCG/data/MiSeq_amplicon/PlotData.txt', header = TRUE)
colnames(plantData)

 [1] "Unique"             "PlotID"             "Plant"             
 [4] "Plot"               "Treatment"          "Rep"               
 [7] "DAP"                "Sampling"           "PlantSample"       
[10] "Type"               "Grass_Broad"        "Status"            
[13] "Subfamily"          "Symbol_class"       "Plant_num"         
[16] "Leaf_AreaBiomass_g" "Biomass_T"          "Biomass_P"         
[19] "Leaf_Area_cm2"      "SLA"                "Leaf_N"            
[22] "Leaf_C"             "LeafC_N"            "LNa"               
[25] "Plant_N"            "Plant_C"            "Plant_CN"          
[28] "d15N"               "tN_p"               "SoilID"            
[31] "Moisture"           "PO_Activity"        "PER_Activity"      
[34] "PO_barefactor"      "PER_barefactor"     "PO_dif"            
[37] "PER_dif"            "PO_rel"             "PER_rel"           
[40] "BX_act"             "CB_act"             "LAP_act"           
[43] "NAG_act"            "BX_activity_g"      "

In [36]:
%%R
s = sample_data(physeq.thresh)
PD = plantData[,c(1,18, 20,21, 23:29, 44:47, 52:59)]
s2 = left_join(s, PD, by = c("X.sample" = "Unique"))
rownames(s2) = s2$X.sample
sample_data(physeq.thresh) = s2
sample_data(physeq.thresh) %>% head

Sample Data:        [6 samples by 44 sample variables]:
                           X.sample Library Primer_Plate Primer_Well_ID
sample2-1.06_Npos sample2-1.06_Npos       1            1             H9
sample6-4.27_Npos sample6-4.27_Npos       1            2            F11
sample4-4.27_Nneg sample4-4.27_Nneg       1            1            B12
sample4-1.06_Nneg sample4-1.06_Nneg       1            1            E11
sample2-1.06_Nneg sample2-1.06_Nneg       1            1             G9
sample6-3.07_Nneg sample6-3.07_Nneg       1            2            C11
                  primerFR_ID_byPlate primerFR_ID_total barcode_ID    PlotID
sample2-1.06_Npos                  72                72         72 1.06_Npos
sample6-4.27_Npos                  86               182        182 4.27_Npos
sample4-4.27_Nneg                  90                90         90 4.27_Nneg
sample4-1.06_Nneg                  85                85         85 1.06_Nneg
sample2-1.06_Nneg                  71                71

# Output for other analyses

In [37]:
%%R -i workDir
outFile = file.path(workDir, 'Full-Sparsity3in3')
saveRDS(physeq.thresh, outFile)